# Noteboook for t2max verification with climate statistics
## Verification of deterministic forecasts using climate percentiles
(requires to run notebook_get_forecast_single.ipynb first if you want to assess your own set of forecasts)


Assessing precipitation forecasts against observations using a local climatology for score computation. 

In [1]:
import time
import datetime
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
from utils.utils_data  import get_obs
from utils.utils_data  import get_fct

from utils.utils_data  import get_domain
from utils.utils_stats  import get_weights

from utils.utils_scores import get_CT
from utils.utils_scores import get_score
from utils.utils_plots  import plot_scores
from utils.utils_plots  import plot_simple_scores

### 1. Settings

In [3]:
# where the data sit or will sit
path_data     = "/my/data/folder/seeps4all/"
path_data     = "/ec/res4/scratch/mozb/hackathon/t2max"
path_data     = "/perm/mozb/RODEO/SEEPS4ALL/DATA_TX/"

# forecasts
name_forecasts = [
                  "fct_od_0001_t2max_europe_20231201_to_20240229_1200_ecad",
                  "fct_ml_aifs_t2max_europe_20231201_to_20240229_1200_ecad",
                  "fct_ml_dmgc_t2max_europe_20231201_to_20240229_1200_ecad",
                  "fct_ml_pguw_t2max_europe_20231201_to_20240229_1200_ecad"
                 ]
#name_forecasts = [
#                  "fct_od_0001_t2max_europe_20240301_to_20240531_1200_ecad",
#                  "fct_ml_aifs_t2max_europe_20240301_to_20240531_1200_ecad",
#                  "fct_ml_dmgc_t2max_europe_20240301_to_20240531_1200_ecad",
#                  "fct_ml_pguw_t2max_europe_20240301_to_20240531_1200_ecad"
#                  "fct_od_0001_t2max_europe_20240601_to_20240831_1200_ecad",
#                 ]
#name_forecasts = [
#                  "fct_ml_aifs_t2max_europe_20240601_to_20240831_1200_ecad",
#                  "fct_ml_dmgc_t2max_europe_20240601_to_20240831_1200_ecad",
#                  "fct_ml_pguw_t2max_europe_20240601_to_20240831_1200_ecad"
#                 ]
#name_forecasts = [
#                  "fct_od_0001_t2max_europe_20240901_to_20241130_1200_ecad",
#                  "fct_ml_aifs_t2max_europe_20240901_to_20241130_1200_ecad",
#                  "fct_ml_dmgc_t2max_europe_20240901_to_20241130_1200_ecad",
#                  "fct_ml_pguw_t2max_europe_20240901_to_20241130_1200_ecad"
#                 ]

# colours and names 
colors = ["tab:red","darkred","tab:blue","tab:purple"]
labels = ["IFS", "AIFS", "GraphCast", "PanguWeather" ]

# ouput path and prefix
prefig = "./prefix"


### 2. Read observations (+ metadata, + climate) and forecasts

In [4]:
# open data
obs_data = get_obs(path_data,"clim",obs_origin="ecad",param="t2max")
fct_data = get_fct(path_data,name_forecasts)

open: /perm/mozb/RODEO/SEEPS4ALL/DATA_TX//obs_clim_t2max_2022_2024_ecad.zarr
... total number of observation locations: 5165
. get data from https://object-store.os-api.cci2.ecmwf.int


FileNotFoundError: No such file or directory: 'https://object-store.os-api.cci2.ecmwf.int/ecmwf-rodeo-benchmark/seeps4all/fct_od_0001_t2max_europe_20231201_to_20240229_1200_ecad.zarr'

### 2B exploring the data

In [ ]:
lat = obs_data.lat.values
lon = obs_data.lon.values
#plt.plot(lon,lat,".")

In [ ]:
from mpl_toolkits.basemap import Basemap, cm
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_0,lat_0=90.,lat_ts=lat_0,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[2],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[2],\
            rsphere=6371200.,resolution='l',area_thresh=10000)
# draw coastlines, state and country boundaries, edge of map.
m.drawcoastlines()
m.drawstates()
m.drawcountries()
plt.scatter(lon,lat,c=lat,cmap="bwr",vmin=-5,vmax=5)
plt.xlim([-15,45])
plt.ylim([35,75])
plt.colorbar()
plt.show()
asd

In [ ]:
# scatter plot
iex = 0
istep = 5
if True:
    runs = fct_data[iex].run.values
    steps = fct_data[iex].step.values

    print(f"experiment {iex}, {len(steps)} steps")
        
    vtime = [pd.Timestamp(vt)-pd.Timedelta(24,unit="h") for vt in runs+steps[istep] ]
    valid_t = ["%s-%02d-%02d"%(vt.year,vt.month,vt.day) for vt in vtime]

    # select observations
    o = obs_data.sel(time=valid_t)

    # select ensemble
    f = fct_data[iex].sel(step=steps[istep])

f0 = f.forecast.values[0,:]
o0 = o.observation.values[0,:]
plt.plot(o0,f0,'.')

### 3. Domain and station weighting 

In [ ]:
# verification domain
domain = "europe" # default otherwise define
obs_data,fct_data = get_domain(obs_data,fct_data,domain)

In [ ]:
# weighting: "uniform" or "station_weighting"
weights = get_weights(obs_data,"station_weighting")

### 4.a Compute RMSE

In [ ]:
plot_info= dict()
plot_info["prefig"] = prefig
plot_info["colors"] = colors
plot_info["labels"] = labels

In [ ]:
rmse = get_score(obs_data,fct_data,weights,"me")
plot_simple_scores(rmse,"bias",plot_info,bootstrap=False)

In [ ]:
rmse = get_score(obs_data,fct_data,weights,"rmse")
plot_simple_scores(rmse,"RMSE",plot_info,bootstrap=False)

### 4.b Compute  threshold dependent scores

Compute PSS, ETS, FBI for a given threshold over a verification period for each forecast lead time

In [ ]:
# percentiles used as thresholds
thresholds = ("perc65","perc70","perc75","perc80","perc85","perc90","perc95","perc98","perc99") 
#get contingency tables
CT = get_CT(obs_data,fct_data, weights,thresholds)

In [ ]:
# info for plotting
plot_info= dict()
plot_info["thresholds"] = thresholds
plot_info["prefig"] = prefig
plot_info["colors"] = colors
plot_info["labels"] = labels

Plots as a function of the lead time

In [ ]:
plot_scores(CT,"FBI",plot_info,along="step",x_list=[0,2,6,8],bootstrap=False)

In [ ]:
plot_scores(CT,"ETS",plot_info,along="step",x_list=[2,6,8],bootstrap=True)

In [ ]:
#plot_scores(CT,"ETS difference",plot_info,along="step",x_list=[0,2,6,8])

In [ ]:
#plot_scores(CT,"ETS relative gain",plot_info,along="step",x_list=[0,2,6,8])

In [ ]:
plot_scores(CT,"PSS",plot_info,along="step",x_list=[0,2,6,8])

In [ ]:
#plot_scores(CT,"PSS difference",plot_info,along="step",x_list=[0,2,6,8])

Plots as a function of the threshold

In [ ]:
plot_scores(CT,"FBI",plot_info,along="thresholds",x_list=[0,2,4],bootstrap=False)

In [ ]:
plot_scores(CT,"ETS",plot_info,along="thresholds",x_list=[0,2,3,4],bootstrap=False)

In [ ]:
#plot_scores(CT,"ETS difference",plot_info,along="thresholds",x_list=[0,2,4])

In [ ]:
#plot_scores(CT,"ETS skill",plot_info,along="thresholds",x_list=[0,2,4])

In [ ]:
#plot_scores(CT,"ETS relative gain",plot_info,along="thresholds",x_list=[0,2,4])

In [ ]:
plot_scores(CT,"PSS",plot_info,along="thresholds",x_list=[0,2,4])

In [ ]:
#plot_scores(CT,"PSS difference",plot_info,along="thresholds",x_list=[0,2,4])

In [ ]:
#plot_scores(CT,"PSS relative gain",plot_info,along="thresholds",x_list=[0,2,4])

End of the Notebook